<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Выводы-по-первому-шагу:" data-toc-modified-id="Выводы-по-первому-шагу:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Выводы по первому шагу:</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
pip install pymystem3

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install phik

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [4]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import f1_score
from scipy import stats as st
from math import factorial
from sklearn.svm import SVR
from sklearn.preprocessing import LabelEncoder
import phik
from pymystem3 import Mystem

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import binom
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
import math
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
import re
import nltk
from sklearn import metrics

from sklearn.model_selection import GridSearchCV
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
#import kmeans
from sklearn.cluster import KMeans
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from nltk.corpus import stopwords 
import warnings
warnings.filterwarnings("ignore")
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier

In [6]:
import spacy


In [7]:
try:
    data=pd.read_csv('/Users/konstantingrigorev/Desktop/toxic_comments.csv', index_col = 0)
except:
    data=pd.read_csv('/datasets/toxic_comments.csv', index_col = 0)

Загружаем наши датасеты

In [8]:
data

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159446,""":::::And for the second time of asking, when ...",0
159447,You should be ashamed of yourself \n\nThat is ...,0
159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159449,And it looks like it was actually you who put ...,0


Текст на английском языке, надо скачать пакеты английского языка для стоп слов, так же замечаем, что присутствуют раздлелители строчек /n, их надо будет в будущем убрать

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


Пропусков нету, типы данных выставленны правильно, всё хорошо.

In [10]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
stopwords = set(stopwords.words('english'))


In [12]:

def data_stop(text):
    text = re.sub(r"(?:\n|\r)", " ", text)
    text = re.sub(r"[^a-zA-Z ]+", "", text).strip()
    text = text.lower()
    return text

data['text'] = data['text'].apply(data_stop)

Создаем функцию для очистки текста

In [13]:
data.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,daww he matches this background colour im seem...,0
2,hey man im really not trying to edit war its j...,0
3,more i cant make any real suggestions on impro...,0
4,you sir are my hero any chance you remember wh...,0


In [14]:
#m = Mystem()
#corpus = data['text'].values.astype('U')
#def lemmatize(text):
#    lemm = m.lemmatize(text)
 #   return "".join(lemm)

#corpus[0] = lemmatize(corpus[0])
#from pymystem3 import Mystem

#m = Mystem()

#def lemmatize(text):
 #   lemm = m.lemmatize(text)
 #   return "".join(lemm)

#corpus = data['text'].values

#lemmatized_corpus = [lemmatize(text) for text in corpus]


In [15]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')


#lemmatizer = WordNetLemmatizer()
#corpus = data['text']
#def lemmatize(text):
 #   words = word_tokenize(text)
 #   lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
 #   lemmatized_text = ' '.join(lemmatized_words)
 #   return lemmatized_text
#lemmatized_corpus = [lemmatize(text) for text in corpus]


[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [16]:
def lemmatize_with_wordnet(text):
    tokens = word_tokenize(text)
    
    lemmatizer = WordNetLemmatizer()
    
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in tokens]
    
    lemmatized_text = ' '.join(lemmatized_tokens)
    
    return lemmatized_text

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN) 

data['lemmatized_text'] = data['text'].apply(lemmatize_with_wordnet)

data.head()

,text,toxic,lemmatized_text
0,explanation why the edits made under my userna...,0,explanation why the edits make under my userna...
1,daww he matches this background colour im seem...,0,daww he match this background colour im seemin...
2,hey man im really not trying to edit war its j...,0,hey man im really not try to edit war it just ...
3,more i cant make any real suggestions on impro...,0,more i cant make any real suggestion on improv...
4,you sir are my hero any chance you remember wh...,0,you sir be my hero any chance you remember wha...


### Выводы по первому шагу:
- При анализе пропусков данных не обнаружено
- Текст очищен от разделителя строк /n
- Текст лематизирован и готов к обучению

## Обучение

In [17]:

X = data['text']
y = data['toxic'].values

X_train_other, X_test, y_train_other, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_other, y_train_other, test_size=0.25, shuffle=False, random_state=42)



Разделяем выборку

In [18]:

count_tf_idf = TfidfVectorizer(stop_words='english')

tfidf_train = count_tf_idf.fit_transform(X_train)
tfidf_valid = count_tf_idf.transform(X_valid)
tfidf_test = count_tf_idf.transform(X_test)


выполняем TF-IDF

In [19]:

pipe_lr = Pipeline([
    ('model', LogisticRegression(random_state=1, solver='liblinear', max_iter=200))
])

param_grid_lr = {
    'model__penalty': ['l1', 'l2'],
    'model__C': list(range(1, 15, 3))
}

grid_lr = GridSearchCV(pipe_lr, param_grid=param_grid_lr, scoring='f1', cv=3, verbose=True, n_jobs=-1)

best_model_lr = grid_lr.fit(tfidf_train, y_train)
print('Лучшие параметры для LogisticRegression:', grid_lr.best_params_)
print('Лучший результат LogisticRegression:', grid_lr.best_score_)

param_grid_rf = {
    'n_estimators': list(range(50, 300, 50)),
    'max_depth': [5, 15],
    'max_features': list(range(1, 20, 2))
}

grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid=param_grid_rf, scoring='f1', cv=3, verbose=True, n_jobs=-1)

best_model_rf = grid_rf.fit(tfidf_train, y_train)
print('Лучшие параметры для  RandomForestClassifier:', grid_rf.best_params_)
print('Лучший результат RandomForestClassifier:', grid_rf.best_score_)

model_cb = CatBoostClassifier(random_state=42, verbose=0)
model_cb.fit(tfidf_train, y_train)
valid_pred_cb = model_cb.predict(tfidf_valid)
f1_score_cb = f1_score(y_valid, valid_pred_cb)

print('Лучший результат CatBoostClassifier:', f1_score_cb)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Лучшие параметры для LogisticRegression: {'model__C': 4, 'model__penalty': 'l1'}
Лучший результат LogisticRegression: 0.7618286242428897
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Лучшие параметры для  RandomForestClassifier: {'max_depth': 5, 'max_features': 1, 'n_estimators': 50}
Лучший результат RandomForestClassifier: 0.0
Лучший результат CatBoostClassifier: 0.7530492634246793


Модель логистической регрессии показала относительно хороший показатель F1 равный 0,766 на валидационной выборке. Это указывает на то, что модель довольно хорошо справляется с предсказанием токсичности на основе текстовых признаков
Случайны лес показал плохие результаты на этой задаче, как указывает показатель F1 равный 0,0 на валидационной выборке
CatBoost показал хороший показатель F1 равный 0,751 на валидационной выборке, что указывает на сильную производительность в предсказании токсичности

In [20]:
test_pred_lr = best_model_lr.predict(tfidf_test)
f1_score_lr_test = f1_score(y_test, test_pred_lr)
print('Logistic Regression Test Метрики:')
print(metrics.classification_report(y_test, test_pred_lr))
print('F1 результат для Logistic Regression на Test Data:', f1_score_lr_test)

#test_pred_cb = model_cb.predict(tfidf_test)
#f1_score_cb_test = f1_score(y_test, test_pred_cb)
#print('\nCatBoost Classifier Test метрики:')
#print(metrics.classification_report(y_test, test_pred_cb))
#print('F1 результат для CatBoost Classifier на Test Data:', f1_score_cb_test)

Logistic Regression Test Метрики:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     14314
           1       0.86      0.72      0.78      1616

    accuracy                           0.96     15930
   macro avg       0.91      0.85      0.88     15930
weighted avg       0.96      0.96      0.96     15930

F1 результат для Logistic Regression на Test Data: 0.7827841291190316


## Выводы

Вот результаты настройки моделей с использованием кросс-валидации:

Для модели логистической регрессии:

Лучшие параметры: {'model__C': 4, 'model__penalty': 'l1'}
Лучший результат (средняя F1-мера на кросс-валидации): 0.7658
Случайный лес:

Лучшие параметры: {'max_depth': 5, 'max_features': 1, 'n_estimators': 50}
Лучший результат (средняя F1-мера на кросс-валидации): 0.0

Для модели CatBoost Classifier:

Лучший результат (F1-мера на валидационном наборе данных): 0.7511

Модель логистической регрессии имеет наилучший средний результат F1-меры на кросс-валидации (0.7658). Это означает, что наши данные лучше всего соответствуют линейной модели, чем моделям случайного леса и CatBoost.